In [1]:
import linguamind.linalg as la
import linguamind.nn as nn

In [2]:
class Linear():
    def __init__(self, input_dim, output_dim):
        
        self.sparse_output = False
        self.sparse_input = False
        
        self.input_dim = input_dim # embedding dim
        self.output_dim = output_dim # output vocab size 
        
        self.weights = la.Matrix(output_dim,input_dim)
        
        self.output = la.Vector(output_dim).zero()
        self.input_grad = la.Vector(self.input_dim).zero()
        
        self.full_output_indices = list(range(self.output_dim))
    
    def updateOutput(self, input, output_indices=None):
        for index in range(self.output_dim):
            self.output.doti(index,input, self.weights[index])
    
    def updateInputGrad(self, output_grad):
        self.input_grad.set(self.weights[0],output_grad[0]) 
        for index in range(self.output_dim-1):
            self.input_grad.addi(self.weights[index+1],output_grad[index+1])
            
    def accGradParameters(self, input, output_grad, alpha):
        for index in range(self.output_dim):
            self.weights[index].addi(input,output_grad[index] * -alpha)



In [3]:
class SparseLinearInput():
    
    def __init__(self,input_dim, output_dim):
        
        self.sparse_output = False
        self.sparse_input = True
        
        self.input_dim = input_dim # vocab size
        self.output_dim = output_dim # embedding dim
        
        self.weights = la.Matrix(input_dim,output_dim)
        
        self.input_indices = list(range(self.input_dim))
        self.full_output_indices = list(range(self.output_dim))        
        
        self.output = la.Vector(output_dim).zero()
        
        self.input_grad = None;
        
    def updateOutput(self, input, input_indices):
        self.input_indices = input_indices
        
        self.output.zero()
        for index in self.input_indices:
            self.output += self.weights[index]
    
    def updateInputGrad(self, output_grad):
        "do nothing"
    
    def accGradParameters(self, input, output_grad, alpha):
        # input is all 1s and is not used
        for index in self.input_indices:
            self.weights[index].addi(output_grad,-alpha)

In [4]:
class SparseLinearOutput():
    
    def __init__(self, input_dim, output_dim):
        
        self.sparse_output = True
        self.sparse_input = False
        
        self.input_dim = input_dim # embedding dim
        self.output_dim = output_dim # output vocab size 
        
        self.weights = la.Matrix(output_dim,input_dim)
        
        self.output = la.Vector(output_dim).zero()
        self.output_indices = list(range(self.output_dim))
        self.full_output_indices = list(range(self.output_dim))        
        
        self.input_grad = la.Vector(self.input_dim).zero()
    
    def updateOutput(self, input, output_indices=None):
        if(output_indices is None):
            output_indices = list(range(self.output_dim))
        self.output_indices = output_indices
        
        for index in self.output_indices:
            self.output.doti(index,input, self.weights[index])
            
    def updateInputGrad(self,output_grad):
        self.input_grad.set(self.weights[self.output_indices[0]],output_grad[self.output_indices[0]]) 
        for index in self.output_indices[1:]:
            self.input_grad.addi(self.weights[index],output_grad[index]) 
    
    def accGradParameters(self, input, output_grad, alpha):
        for index in self.output_indices:
            self.weights[index].addi(input,output_grad[index] * -alpha)


In [5]:
class MSECriterion():
    
    def __init__(self):
        ""
        self.grad = la.Vector(1).zero()
        
    def forward(self,input,target,output_indices):
        self.output = input
        self.tmp_error = la.Vector(len(output_indices)).zero()
        for i,index in enumerate(output_indices):
            self.tmp_error[i] = self.output[index] - target[i]
            
        self.error = 0
        for i in range(len(output_indices)):
            self.error += self.tmp_error[i] * self.tmp_error[i]
        self.error /= len(self.tmp_error)
        
        return self.error
    
    def backward(self,output, target,output_indices):
        if(self.grad.size != output.size):
            self.grad = la.Vector(output.size).zero()
        for i,index in enumerate(output_indices):
            self.grad[index] = output[index] - target[i]
        return self.grad

In [6]:
class Sequential():
    
    def __init__(self):
        ""
        self.layers = list()

    def get(self,i):
        return self.layers[i]
    
    def add(self,layer):
        
        self.layers.append(layer)
        self.output = self.layers[-1].output
    
    def forward(self,input_indices=(1,3,2), output_indices=(1,2,4)):
        
        self.layers[0].updateOutput(None, input_indices)
        
        sparse_output_until_end = False
        
        
        for index in range(len(self.layers)-2):
            if(self.layers[index+1].hasSparseOutput()):
                sparse_output_until_end = True
                
            if(sparse_output_until_end):
                self.layers[index+1].updateOutput(self.layers[index].getOutput(),output_indices)
            else:
                self.layers[index+1].updateOutput(self.layers[index].output,self.layers[index+1].getFullOutputIndices())
            
        self.layers[-1].updateOutput(self.layers[-2].getOutput(),output_indices)
        
        return self.output
    
    def backward(self, grad, output_indices):
        
        self.layers[-1].updateInputGrad(grad)
        sparse_output_until_end = True
        for i in reversed(range(len(self.layers)-2)):
            if(sparse_output_until_end):
                self.layers[i+1].updateInputGrad(self.layers[i+2].input_grad)
            else:
                self.layers[i+1].updateInputGrad(self.layers[i+2].input_grad)
            if(self.layers[i+1].sparse_output == True):
                sparse_output_until_end = False

In [7]:
class Relu(nn.Layer):
    def __init__(self, dim):

        self.sparse_output = False
        self.sparse_input = False
    
        self.input_dim = dim
        self.output_dim = dim
        
        self.weights = None
        self.output_indices = list(range(dim))
        
        self.output = la.Vector(self.output_dim).zero()
        self.input_grad = la.Vector(self.input_dim).zero()
        
        self.full_output_indices = list(range(self.output_dim))
    
    def updateOutput(self, input, output_indices = None):

        if(output_indices is None):
            output_indices = list(range(self.output_dim))
        self.output_indices = output_indices
        
        self.input_grad *= 0
        self.input_grad += input
        self.input_grad >= 0
        
        self.output *= 0
        self.output += input
        self.output *= self.input_grad
        
    def updateInputGrad(self, output_grad):
        self.input_grad *= output_grad
        
    def accGradParameters(self,input, output_grad, alpha):
        "do nothing"

In [8]:
class StochasticGradient():
    
    def __init__(self,mlp,criterion,alpha = 0.0001):
        self.mlp = mlp
        self.criterion = criterion
        self.alpha = alpha
    
    def train(self,input_indices, output_indices, target_values): 
        
        pred = self.mlp.forward(input_indices,output_indices)
        error = self.criterion.forward(pred,target_values,output_indices)
        self.mlp.backward(self.criterion.backward(pred,target_values,output_indices),output_indices)

        # update all but last layer
        prev_layer_output = None        
        for i in range(len(layers)-1):
            mlp.get(i).accGradParameters(prev_layer_output,mlp.get(i+1).getInputGrad(),self.alpha)
            prev_layer_output = mlp.get(i).getOutput()
                    
        # update weights for output with criterion gradient        
        mlp.get(len(layers)-1).accGradParameters(mlp.get(len(layers)-2).getOutput(),criterion.grad,self.alpha)
        return error


In [9]:
seed = la.Seed(1)

syn0 = nn.SparseLinearInput(100,64)
syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= 50

syn1 = nn.Linear(64,32)
syn1.weights.uniform(seed)
syn1.weights -= 0.5
syn1.weights /= 50

syn2 = nn.Linear(32,16)
syn2.weights.uniform(seed)
syn2.weights -= 0.5
syn2.weights /= 50

syn3 = nn.SparseLinearOutput(16,100)
syn3.weights.uniform(seed)

layers = list()
layers.append(syn0)
layers.append(syn1)
layers.append(nn.Relu(32))
layers.append(syn2)
layers.append(nn.Relu(16))
layers.append(syn3)
layers.append(nn.Relu(100))

In [10]:
mlp = nn.Sequential(layers)

criterion = nn.MSECriterion()

In [11]:
input_indices = (21,22,23,24,26,27,28)
output_indices = (11,12,13,14,15,16,10,14)

target = la.Vector(len(output_indices))
target.zero()
target.set(0,1)

In [12]:
optim = nn.StochasticGradient(mlp,criterion)

In [15]:
%%timeit
optim.train(input_indices,output_indices,target,alpha=0.001,iterations=1000)

100 loops, best of 3: 7.87 ms per loop


In [15]:
for iter in range(1000):
    print(optim.train(input_indices,output_indices,target,alpha=0.01))

0.12497355043888092
0.1249733716249466
0.12497317790985107
0.12497298419475555
0.12497280538082123
0.12497261166572571
0.12497241795063019
0.12497222423553467
0.12497203052043915
0.12497183680534363
0.12497164309024811
0.1249714344739914
0.12497124075889587
0.12497104704380035
0.12497083842754364
0.12497064471244812
0.1249704360961914
0.12497024238109589
0.12497003376483917
0.12496982514858246
0.12496961653232574
0.12496940791606903
0.12496919929981232
0.1249689906835556
0.12496878206729889
0.12496855854988098
0.12496834993362427
0.12496812641620636
0.12496791779994965
0.12496769428253174
0.12496748566627502
0.12496726214885712
0.12496703863143921
0.1249668151140213
0.1249665915966034
0.12496637552976608
0.12496613711118698
0.12496591359376907
0.12496569007635117
0.12496545165777206
0.12496522814035416
0.12496498972177505
0.12496475130319595
0.12496451288461685
0.12496427446603775
0.12496403604745865
0.12496379762887955
0.12496356666088104
0.12496331334114075
0.12496307492256165
0.1249

In [ ]:
for iter in range(1000):
    print(optim.train(input_indices,output_indices,target))

In [39]:
n_sample = 10

In [40]:
syn0 = nn.SparseLinearInput(1000,50)
syn0.weights.uniform()
syn0.weights -= 0.5
syn0.weights /= 50

syn1 = nn.SparseLinearOutput(50,1000,n_sample)
syn1.weights.uniform()

<linguamind.linalg.Tensor; proxy of <Swig Object of type 'Tensor *' at 0x1078b4e70> >

In [41]:
mlp = nn.Sequential()
mlp.add(syn0)
mlp.add(syn1)

criterion = nn.MSECriterion(1,n_sample)

In [42]:
target = la.Tensor((1,10))
target.zero()
target.set(0,1)

In [43]:
context_indices = (1,2,4,5)
pred = 3
neg_samples = list([6,7,8,9,10,11,12,13,14])
target_indices = list([pred]) + neg_samples

In [44]:
criterion.forwards(mlp.forward(context_indices,target_indices))

In [45]:
criterion.backwards(target)

<linguamind.linalg.Tensor; proxy of <Swig Object of type 'Tensor *' at 0x1078b4630> >

In [ ]:
input = la.Tensor((1,50)).uniform()

In [ ]:
syn1.updateOutput(input,(1,3))

In [ ]:
syn1.output.get(1)

In [ ]:
text = nlp.Text("hello world","BIIIIOBIIII")

In [ ]:
vocab = text.getVocab("tokens")

In [ ]:
text.getSequence('tokens')

In [ ]:
# text = lm.Text("hello world")

In [ ]:
text = lm.Text("hello world","BIIIIOBIIII")

In [ ]:
vocab = text.getVocab("tokens")

In [ ]:
for i in xrange(vocab.size):
    print vocab.getTermAtIndex(i).letters

In [ ]:
text.getSequence("tokens")

#NLP Components

Two Types of Component:
- word level
- sentence level
- document level

### Data Model

In [ ]:
c = lm.Text("/my/folder/of/files") # atempts to autodetect file type
c = lm.Text(path="myfile.txt",type="raw")
c = lm.Text(path="myfile.txt",type="csv-token") # one token per line in CSV
c = lm.Text(path="myfile.txt",type="csv-segment") # text segment per line in CSV
c = lm.Text(path="myfile.txt",type="prefix__label__") # one sentence per line with special labels
c = lm.Text(path="myfile.txt",type="json")

# document level raw text
d1 = lm.Text("This is my sentence.")
d2 = lm.Text("myfile.txt")

c = lm.Text([d1,d2]) #form documents into Corpus

# document level text with labels
contents = {}
contents["text"] = "This is my sentence. This is another one."
contents["date"] = 1997
d = lm.Text(contents)

# sentence level text with labels
contents = {}
contents["text"] = "This is my sentence."
contents["sentiment"] = "Positive"
contents["speaker"] = "John Locke"
d = lm.Text(contents)

### Applying Analytics

In [ ]:
tok = lm.Segmenter(name="tokenizer",lang="en")
eos = lm.Segmenter(name="eos",lang="en")

# document level text with labels
contents = {}
contents["text"] = "This is my sentence. This is another one."
contents["date"] = 1997
d = lm.Text(contents)

tok.segment(d)
eos.segment(d)

print d["text"] # "This is my sentence. This is another one."
print d["tokens"] # ["This", "is"...
print d["sentences"] # [["This", "is", "my", "sentence","."],["This","is","another","one","."]]                
                     
pos = lm.Classifier(name="pos")
pos.predict(d)

print d["sentences"] # [[{"token":"This","POS":"NNP"},...                

sentiment = lm.Classifier(name="sentiment")
sentiment.predict(d)
print d["sentences"] # [{"sentiment":"Positive","tokens":[{"token":"This","POS":"NNP"},...

###Creating Analytics

In [ ]:
# Creating a Segmenter (tokenizer)
contents = {}
contents["text"] = "I like pie."
contents["char_segs"] = "BOBIIIOBIIB"
d = lm.Text(contents)

feats=[["text_-2","text_-1"],"text_-1","text_0","text_1",["text_1","text_2"]]

ambiguity_hash = lm.AmbiguityHash(d,feats,"char_segs",threshold=0.95,min_count=5)

percept_model = lm.ml.Sequential()
percept_model.add(lm.ml.Linear(size=[d.getVocab("text").size() * 10],encoding="hashtable"))
percept_model.add(lm.ml.LogSoftmax())

percept_loss = lm.ml.PerceptronLoss()
percept_optimizer = lm.ml.optim.PerceptronUpdate()
percept_searcher = lm.ml.search.BeamSearch(beam=5)

# where available in leau of calling all the forward/backprop logic from Python slowing things down
tokenizer = lm.ml.trainer.FastSegmentationTrainer(d,"char_segs",feats,percept_model,percept_loss,percept_searcher,inherits=[ambiguity_hash],threads=50)

tokenizer.segment(d,"tokens") #generates new vocab for what was segmented

eos = lm.pretrained.Segmenter(name="eos",lang="en")
eos.segment(d,"sentences",ignore_vocab=True) # does not generate a new vocab for what was segmented

# https://github.com/oxford-cs-ml-2015/practical6/blob/master/train.lua
lstm_hidden = 50
seq_length = 16

layers = list()
for i in range(seq_length):
    layer = {}
    
    inputs = {}
    layer['embed'] = lm.ml.Embedding(d.getVocab("token").size(),lstm_hidden)
    layer['lstm'] = lm.ml.LSTM(lstm_hidden,input_x=layer['embed'],input_h=layers[i-1]['lstm']['h'])
    layer['softmax'] = lm.ml.LogElasticHierarchicalSoftmax(input=layer['lstm']['h'],size=(lstm_hidden,d.getVocab("token").size()),sample_rate=0.001)
    layer['criterion'] = lm.ml.LogElasticHierarchicalSoftmaxLoss()
    layers.append(layer)

word_language_model_searcher = lm.ml.search.BeamSearch(beam=5)
percept_optimizer = lm.ml.optim.SGD()
word_language_model = lm.ml.trainer.FastLanguageModelTrainer(d,layers,word_language_model_searcher,percept_optimizer,predict_on="token",bound_on="sentence",threads=50)

#ML Components

In [ ]:
#Ambiguity Hashing

In [ ]:
#Perceptron 

In [ ]:
#LSTM 

In [ ]:
#Neural Index